In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

In [2]:
scenarios = [
    #{"model": "image", "pathway": "SSP2-Base", "year": 2020},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2022},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2025},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2030},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2040},
    {"model": "image", "pathway": "SSP2-Base", "year": 2050},
]

sps = Package("../datapackage.json")

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[sps,],
        use_multiprocessing=False
)

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")
sps = Package("../datapackage.json")
ndb = PathwaysDataPackage(
    scenarios=[
        {"model": "image", "pathway": "SSP2-Base"},
    ],
    #years=[2005, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100],
    years=[2020, 2022, 2025, 2030, 2035, 2040, 2045, 2050],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    external_scenarios=[sps,]
)

OperationalError: database is locked

In [ ]:
ndb.create_datapackage(
    name="image-SSP2-stem-SPS1",
    contributors=[
        {"name": "Romain",
        "email": "r_s at me.com",}
    ]
)

In [1]:
from pathways import Pathways
p = Pathways(datapackage="image-SSP2-stem-SPS1/datapackage.json")

Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/17" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/17/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Electricity not found in scenario data.
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Gases|Natural Gas not found in scenario data.
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Heat not found in scenario data.
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Hydrogen not found in scenario data.
Variable Final Energy|Industry 

In [2]:
import numpy as np
p.calculate(
    methods=[
        #'IPCC 2021 - climate change - GWP 100a, incl. H',
        'IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2',
    ],
    regions=["CH",],
    scenarios=[
        "SSP2-Base",
        #"SSP2-RCP19",
    ],
    variables=[
        "hydro_run_of_river",
        "hydro (reservoir)",
        "hydro (pumped storage)",
        "nuclear (boiling water)",
        "nuclear (pressure water)",
        "oil (peak devices)",
        "oil (DH CHP)",
        "oil (industry CHP)",
        "gas (CCGT)",
        "gas (OCGT)",
        "gas (CCGT CCS)",
        "gas (CHP DH)",
        "gas (fuel cell DH)",
        "gas (CHP industry)",
        "gas (CHP CCS industry)",
        "gas (CHP fuel cell reformer)",
        "gas (CHP single family)",
        "gas (single family CHP fuel cell)",
        "gas (multi family CHP)",
        "gas (multi family CHP fuel cell)",
        "biogas (services CHP)",
        "gas (services CHP)",
        "gas (services CHP fuel cell)",
        "waste incineration, fossil",
        "waste incineration, fossil (CHP)",
        "waste incineration, fossil (industrial)",
        "waste incineration, fossil (CCS)",
        "waste incineration, fossil (CHP, CCS)",
        "waste incineration, fossil (industrial, CCS)",
        "waste incineration, non-fossil",
        "waste incineration, non-fossil (CHP)",
        "waste incineration, non-fossil (industrial)",
        "waste incineration, non-fossil (CCS)",
        "waste incineration, non-fossil (CHP, CCS)",
        "waste incineration, non-fossil (industrial, CCS)",
        "pv (large scale)",
        "pv (industry)",
        "pv (services)",
        "pv residential (multi family)",
        "pv residential (single family)",
        "wind",
        "geothermal",
        "hydrogen fuel cell (DH CHP)",
        "hydrogen fuel cell (industrial CHP)",
        "hydrogen fuel cell (services CHP)",
        "biomass (DH)",
        "biomass (CHP)",
        "biomass (CCS, BECCS)",
        "biomass (CCS, industrial)",
        "battery (large)",
        "battery (medium)",
        "battery (industry)",
        "battery (services)",
        "battery (single family)",
        "battery (multi family)",
        "CAES",
        "import",
    ],
    years=[
        2020,
        2022,
        2030,
        2040,
        2050
    ],
    characterization=True,
    #data_type=np.float32,
    multiprocessing=False,
    demand_cutoff=0.01,
)
arr = p.display_results()

Calculating LCA results for image...
--- Calculating LCA results for SSP2-Base...
------ Calculating LCA results for 2020...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2022...
LCA matrices not found for the given model, scenario, and year.
------ Calculating LCA results for 2030...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2040...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2050...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


Coordinates:
  * model            (model) object 'image'
  * scenario         (scenario) object 'SSP2-Base'
  * year             (year) int64 2020 2022 2030 2040 2050
  * region           (region) object 'CH'
  * impact_category  (impact_category) object 'IPCC 2021 - climate change - G...
  * variable         (variable) object 'CAES' 'battery (industry)' ... 'wind'
  * act_category     (act_category) object 'Aquaculture' ... 'other'
<xarray.DataArray 'year' (year: 5)>
array([2020, 2022, 2030, 2040, 2050])
Coordinates:
  * year     (year) int64 2020 2022 2030 2040 2050
<xarray.DataArray 'year' (year: 5)>
array([2020, 2022, 2030, 2040, 2050])
Coordinates:
  * year     (year) int64 2020 2022 2030 2040 2050


In [11]:
p.lca_results.sel(
    region="CH",
    model="image",
    scenario="SSP2-Base",
    impact_category="IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2",
    variable="biomass (CCS, industrial)"
).sum(dim="act_category").interp(
    year=range(2020, 2050),
    kwargs={"fill_value": "extrapolate"},
            method="linear",
)

<xarray.DataArray (year: 30)>
array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.37807905e+08, -2.75615809e+08, -4.13423714e+08,
       -5.51231619e+08, -6.89039523e+08, -8.26847428e+08, -9.64655333e+08,
       -1.10246324e+09, -1.24027114e+09])
Coordinates:
    variable         <U48 'biomass (CCS, industrial)'
    region           <U2 'CH'
    model            <U5 'image'
    scenario         <U9 'SSP2-Base'
    impact_category  <U58 'IPCC 2021 - climate change - GWP 100a, incl. H and...
  * year             (year) int64 2020 2021 2022 2023 ... 2046 2047 2048 2049

In [22]:
p.scenarios.sel(region="CH", model="image", pathway="SSP2-Base").to_dataframe("x").unstack()["x"]

year,2020,2022,2030,2040,2050
variables,,,,,
hydro_run_of_river,66.6324,55.764,64.462662,62.292915,66.13497
hydro (reservoir),66.74568,49.18294,70.236607,73.020285,69.17823
hydro (pumped storage),12.84192,16.033006,12.550631,12.937638,13.428635
nuclear (boiling water),33.104,33.104,27.973764,13.12832,0.0
nuclear (pressure water),49.656,49.656,41.960646,19.69248,0.0
oil (peak devices),0.0396,0.0,0.000013,0.000057,0.000007
oil (DH CHP),0.0018,0.0,0.000008,0.000015,0.0
oil (industry CHP),0.0705,0.171528,0.171506,0.006626,0.00151
gas (CCGT),0.0,0.0,0.0,0.000539,0.000023


In [4]:
df = arr.to_dataframe("x").unstack()["x"].reset_index().melt(id_vars=[
    "model", "scenario", "region", "impact_category", "variable", "year"
])

In [5]:
df=df.loc[~df["value"].isnull()]

In [6]:
len(df)

2452

In [7]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')

In [6]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')